# Лабораторная работа 5: Seq2Seq-чатбот на RNN (LSTM/GRU)

**Выполнил:** *Лунев Александр Вячеславович*  
**Группа:** *СП1*  

Цель работы — построить и обучить модель Seq2Seq с рекуррентными слоями (LSTM или GRU) 
для генерации ответов на русском языке в формате диалога.

Основные шаги:
1. Загрузка и первичный анализ датасета Toloka Persona Chat (русская версия).
2. Разбор диалогов и формирование пар «реплика → ответ».
3. Токенизация текста, построение словаря и подготовка тензоров.
4. Реализация модели Seq2Seq: кодировщик (encoder) и декодер (decoder) с механизмом внимания.
5. Обучение модели на обучающей выборке и оценка на валидации.
6. Реализация генерации ответов (greedy и beam search).
7. Простой интерактивный чат с обученной моделью.
8. Обсуждение примеров диалогов и выводы.


In [1]:
import os
from pathlib import Path
import random
import math
import re
import collections

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Фиксируем зерно генераторов случайных чисел для воспроизводимости
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# Выбираем устройство (GPU, если доступно, иначе CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используется устройство: {device}")

Используется устройство: cuda


## 1. Загрузка и первичный анализ датасета

В данной работе используется датасет Toloka Persona Chat (русская версия).  



In [2]:
DATA_PATH = "data/dialogues.tsv"

df = pd.read_csv(DATA_PATH, sep="\t")

print("Первые строки датафрейма:")
display(df.head())

print("\nНазвания колонок:")
print(df.columns.tolist())

DIALOGUE_COLUMN = "dialogue"  # при необходимости измените название

if DIALOGUE_COLUMN not in df.columns:
    raise ValueError(
        f"В датафрейме нет колонки '{DIALOGUE_COLUMN}'. "
        f"Доступные колонки: {df.columns.tolist()}. "
        f"Поставьте сюда правильное имя столбца."
    )

print(f"\nКоличество строк в датасете: {len(df)}")

Первые строки датафрейма:


,persona_1_profile,persona_2_profile,dialogue
0,<span class=participant_1>У меня любимая работ...,<span class=participant_2>Ищу принца.<br />Вед...,<span class=participant_2>Пользователь 2: Прив...
1,<span class=participant_1>Я работаю учителем<b...,<span class=participant_2>Я бизнесмен<br />У м...,<span class=participant_1>Пользователь 1: Прив...
2,<span class=participant_1>Я купила дом<br />Я ...,<span class=participant_2>Я пою в караоке<br /...,<span class=participant_1>Пользователь 1: Прив...
3,<span class=participant_1>я врач и женат<br />...,<span class=participant_2>Я мальчик<br />Я учу...,<span class=participant_2>Пользователь 2: Здра...
4,<span class=participant_1>Я школьница.<br />Я ...,<span class=participant_2>Я простоват.<br />Лю...,<span class=participant_1>Пользователь 1: Прив...



Названия колонок:
['persona_1_profile', 'persona_2_profile', 'dialogue']

Количество строк в датасете: 10013


## 2. Разбор диалогов и формирование пар «реплика → ответ»

На этом этапе:
1. Очищаем текст (нижний регистр, удаление HTML-тегов, ссылок и лишних пробелов).
2. Разбиваем диалог на отдельные реплики.
   - В некоторых версиях датасета реплики разделены строками (`\n`),
     в других — разделителем `|||`. Код пытается обработать оба варианта.
3. Фильтруем слишком короткие реплики.
4. Формируем пары вида:  
   *реплика_i → реплика_(i+1)* для всех соседних реплик в диалоге.


In [3]:
MIN_UTTER_LEN = 2  # минимальное количество слов в реплике, чтобы её учитывать


def clean_text(text: str) -> str:
    if not isinstance(text, str):
        text = str(text)
    text = text.lower()
    text = re.sub(r"<[^>]+>", " ", text)
    text = re.sub(r"http\S+|www\.\S+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


def split_dialogue(raw_dialogue: str):
    if not isinstance(raw_dialogue, str):
        return []

    s = raw_dialogue.strip()
    if not s:
        return []
    if "|||" in s:
        parts = s.split("|||")
    elif "\n" in s:
        parts = s.split("\n")
    elif "\\n" in s:
        parts = s.split("\\n")
    else:
        parts = [s]

    parts = [p.strip() for p in parts if p.strip()]
    return parts


def tokenize(text: str):
    return text.split()


def build_pairs(dialogues, min_utter_len: int = MIN_UTTER_LEN, max_pairs: int | None = None):
    pairs = []

    for raw_dialogue in dialogues:
        utterances = split_dialogue(raw_dialogue)
        # очищаем и фильтруем по длине
        utterances = [clean_text(u) for u in utterances]
        utterances = [u for u in utterances if len(tokenize(u)) >= min_utter_len]

        for i in range(len(utterances) - 1):
            src = utterances[i]
            tgt = utterances[i + 1]
            pairs.append((src, tgt))

            if max_pairs is not None and len(pairs) >= max_pairs:
                return pairs

    return pairs


pairs = build_pairs(df[DIALOGUE_COLUMN].tolist(), min_utter_len=MIN_UTTER_LEN)

print(f"Количество сформированных пар 'реплика → ответ': {len(pairs)}")
print("\nПримеры пар:")
for i in range(3):
    print(f"\nПара {i+1}:")
    print(f"  Вход:  {pairs[i][0]}")
    print(f"  Ответ: {pairs[i][1]}")


Количество сформированных пар 'реплика → ответ': 1487

Примеры пар:

Пара 1:
  Вход:  " пользователь 1: привет пользователь 2: привет пользователь 2: как дела? ты откуда? пользователь 1: хорошо! а у тебя? пользователь 1: днепр! а ты? пользователь 2: хорошо, я из ялуторовска, это в россии, тюменская область пользователь 1: далеко
  Ответ: кем работаешь? пользователь 2: работаю на себя))) пользователь 2: а ты? пользователь 1: а я работаю фрилансером пользователь 2: что делаешь в свободное время? пользователь 1: чем занимаешься в свободное время? пользователь 2: :) пользователь 1: саморазвиваюсть, ценю время м не трачу его попусту пользователь 1: а ты? пользователь 2: люблю погулять в парке ... саморазвитие это всегда хорошо пользователь 1: ещё я придумываю что-то ное, чтоб удивить людей пользователь 1: новое* пользователь 2: нравиться делать людей счастливее пользователь 1: именно так "

Пара 2:
  Вход:  " пользователь 2: здравствуйте пользователь 1: привет пользователь 2: как твои дела?

## 3. Токенизация, построение словаря и подготовка тензоров

На этом шаге:
1. Строим словарь по всем словам из входов и ответов.
2. Вводим специальные токены:
   - `<pad>` — добивка до нужной длины;
   - `<bos>` — начало ответа;
   - `<eos>` — конец ответа;
   - `<unk>` — неизвестное слово.
3. Ограничиваем размер словаря (например, 20 000 наиболее частых слов).
4. Преобразуем текстовые реплики в последовательности индексов фиксированной длины
   для входа и выхода:
   - вход: `src_ids`;
   - вход декодера (сдвиг вправо с BOS): `tgt_input_ids`;
   - цель (с EOS в конце): `tgt_output_ids`.


In [9]:
MAX_VOCAB_SIZE = 20000
MAX_LEN_SRC = 20
MAX_LEN_TGT = 20

PAD_TOKEN = "<pad>"
BOS_TOKEN = "<bos>"
EOS_TOKEN = "<eos>"
UNK_TOKEN = "<unk>"

SPECIAL_TOKENS = [PAD_TOKEN, BOS_TOKEN, EOS_TOKEN, UNK_TOKEN]

counter = collections.Counter()
for src, tgt in pairs:
    counter.update(tokenize(src))
    counter.update(tokenize(tgt))

most_common_words = [w for w, c in counter.most_common(MAX_VOCAB_SIZE - len(SPECIAL_TOKENS))]

word2index: dict[str, int] = {}
index2word: dict[int, str] = {}

for idx, token in enumerate(SPECIAL_TOKENS + most_common_words):
    word2index[token] = idx
    index2word[idx] = token

PAD_IDX = word2index[PAD_TOKEN]
BOS_IDX = word2index[BOS_TOKEN]
EOS_IDX = word2index[EOS_TOKEN]
UNK_IDX = word2index[UNK_TOKEN]

vocab_size = len(word2index)
print(f"Размер словаря (включая спец. токены): {vocab_size}")


def text_to_indices(
    text: str,
    word2index: dict[str, int],
    max_len: int,
    add_bos: bool = False,
    add_eos: bool = False,
) -> list[int]:
    tokens = tokenize(text)
    ids: list[int] = []

    if add_bos:
        ids.append(BOS_IDX)

    for tok in tokens:
        ids.append(word2index.get(tok, UNK_IDX))
        if len(ids) >= max_len - int(add_eos):
            break

    if add_eos:
        ids.append(EOS_IDX)

    if len(ids) < max_len:
        ids += [PAD_IDX] * (max_len - len(ids))
    else:
        ids = ids[:max_len]

    return ids


def indices_to_text(indices: list[int], index2word: dict[int, str]) -> str:
    words: list[str] = []
    for idx in indices:
        if idx == PAD_IDX:
            continue
        if idx == BOS_IDX:
            continue
        if idx == EOS_IDX:
            break
        words.append(index2word.get(idx, UNK_TOKEN))
    return " ".join(words)


src_sequences: list[list[int]] = []
tgt_input_sequences: list[list[int]] = []
tgt_output_sequences: list[list[int]] = []

for src, tgt in pairs:
    src_ids = text_to_indices(src, word2index, MAX_LEN_SRC, add_bos=False, add_eos=False)
    tgt_input_ids = text_to_indices(tgt, word2index, MAX_LEN_TGT, add_bos=True, add_eos=False)
    tgt_output_ids = text_to_indices(tgt, word2index, MAX_LEN_TGT, add_bos=False, add_eos=True)

    src_sequences.append(src_ids)
    tgt_input_sequences.append(tgt_input_ids)
    tgt_output_sequences.append(tgt_output_ids)

print(f"Количество последовательностей: {len(src_sequences)}")

# Покажем пример преобразования
example_idx = 0
print("\nПример преобразования:")
print("Исходная реплика: ", pairs[example_idx][0])
print("Ответ:           ", pairs[example_idx][1])
print("src_ids:         ", src_sequences[example_idx])
print("tgt_input_ids:   ", tgt_input_sequences[example_idx])
print("tgt_output_ids:  ", tgt_output_sequences[example_idx])
print("Восстановленный ответ: ", indices_to_text(tgt_output_sequences[example_idx], index2word))


Размер словаря (включая спец. токены): 20000
Количество последовательностей: 1487

Пример преобразования:
Исходная реплика:  " пользователь 1: привет пользователь 2: привет пользователь 2: как дела? ты откуда? пользователь 1: хорошо! а у тебя? пользователь 1: днепр! а ты? пользователь 2: хорошо, я из ялуторовска, это в россии, тюменская область пользователь 1: далеко
Ответ:            кем работаешь? пользователь 2: работаю на себя))) пользователь 2: а ты? пользователь 1: а я работаю фрилансером пользователь 2: что делаешь в свободное время? пользователь 1: чем занимаешься в свободное время? пользователь 2: :) пользователь 1: саморазвиваюсть, ценю время м не трачу его попусту пользователь 1: а ты? пользователь 2: люблю погулять в парке ... саморазвитие это всегда хорошо пользователь 1: ещё я придумываю что-то ное, чтоб удивить людей пользователь 1: новое* пользователь 2: нравиться делать людей счастливее пользователь 1: именно так "
src_ids:          [18, 4, 5, 36, 4, 6, 36, 4, 6, 17, 8

## 4. Разбиение на обучающую и валидационную выборки, DataLoader

Теперь:
1. Делим все пары на обучающую и валидационную части (например, 90% / 10%).
2. Создаём класс `Dataset` для PyTorch.
3. Создаём `DataLoader` для удобной итерации по батчам.


In [10]:
indices = list(range(len(src_sequences)))
random.shuffle(indices)

train_size = int(0.9 * len(indices))
train_indices = indices[:train_size]
val_indices = indices[train_size:]


def select_by_indices(sequences: list[list[int]], idxs: list[int]) -> list[list[int]]:
    return [sequences[i] for i in idxs]


train_src = select_by_indices(src_sequences, train_indices)
train_tgt_in = select_by_indices(tgt_input_sequences, train_indices)
train_tgt_out = select_by_indices(tgt_output_sequences, train_indices)

val_src = select_by_indices(src_sequences, val_indices)
val_tgt_in = select_by_indices(tgt_input_sequences, val_indices)
val_tgt_out = select_by_indices(tgt_output_sequences, val_indices)


class DialogueDataset(Dataset):
    def __init__(
        self,
        src_sequences: list[list[int]],
        tgt_input_sequences: list[list[int]],
        tgt_output_sequences: list[list[int]],
    ):
        assert len(src_sequences) == len(tgt_input_sequences) == len(tgt_output_sequences)
        self.src_sequences = src_sequences
        self.tgt_input_sequences = tgt_input_sequences
        self.tgt_output_sequences = tgt_output_sequences

    def __len__(self) -> int:
        return len(self.src_sequences)

    def __getitem__(self, idx: int):
        src = torch.tensor(self.src_sequences[idx], dtype=torch.long)
        tgt_in = torch.tensor(self.tgt_input_sequences[idx], dtype=torch.long)
        tgt_out = torch.tensor(self.tgt_output_sequences[idx], dtype=torch.long)
        return src, tgt_in, tgt_out


BATCH_SIZE = 64

train_dataset = DialogueDataset(train_src, train_tgt_in, train_tgt_out)
val_dataset = DialogueDataset(val_src, val_tgt_in, val_tgt_out)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Размер обучающей выборки: {len(train_dataset)}")
print(f"Размер валидационной выборки: {len(val_dataset)}")


Размер обучающей выборки: 1338
Размер валидационной выборки: 149


## 5. Определение модели Seq2Seq (энкодер, декодер и внимание)

Здесь реализуем:
1. **Кодировщик (Encoder)** на базе RNN (GRU или LSTM):
   - получает на вход последовательность индексов,
   - возвращает последовательность скрытых состояний и финальное скрытое состояние.
2. **Механизм внимания (Luong attention)**:
   - на каждом шаге декодера вычисляет веса по всем выходам кодировщика,
   - формирует контекстный вектор.
3. **Декодер (Decoder) с вниманием**:
   - на вход получает текущий токен, скрытое состояние и выходы кодировщика,
   - с помощью внимания собирает контекст и предсказывает следующий токен.
4. **Общий класс Seq2Seq**, объединяющий энкодер и декодер.


In [13]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1, dropout=0.1, rnn_type="gru"):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=PAD_IDX)
        rnn_type = rnn_type.lower()
        if rnn_type == "lstm":
            self.rnn = nn.LSTM(
                embed_size,
                hidden_size,
                num_layers=num_layers,
                batch_first=True,
                dropout=dropout,
            )
        else:
            self.rnn = nn.GRU(
                embed_size,
                hidden_size,
                num_layers=num_layers,
                batch_first=True,
                dropout=dropout,
            )
        self.rnn_type = rnn_type

    def forward(self, src):
        embedded = self.embedding(src)  # [batch, src_len, embed_size]
        encoder_outputs, hidden = self.rnn(embedded)
        return encoder_outputs, hidden


class LuongAttention(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.linear_in = nn.Linear(hidden_size, hidden_size, bias=False)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, decoder_hidden, encoder_outputs, mask=None):
        dec = self.linear_in(decoder_hidden)
        dec = dec.unsqueeze(2)

        energies = torch.bmm(encoder_outputs, dec).squeeze(2)

        if mask is not None:
            energies = energies.masked_fill(mask == 0, -1e9)

        attn_weights = self.softmax(energies)
        context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).squeeze(1)
        return context, attn_weights


class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1, dropout=0.1, rnn_type="gru"):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=PAD_IDX)
        self.rnn_type = rnn_type.lower()
        if self.rnn_type == "lstm":
            self.rnn = nn.LSTM(
                embed_size,
                hidden_size,
                num_layers=num_layers,
                batch_first=True,
                dropout=dropout,
            )
        else:
            self.rnn = nn.GRU(
                embed_size,
                hidden_size,
                num_layers=num_layers,
                batch_first=True,
                dropout=dropout,
            )

        self.attention = LuongAttention(hidden_size)
        self.fc_concat = nn.Linear(hidden_size * 2, hidden_size)
        self.fc_out = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.output_size = vocab_size

    def forward(self, input_step, hidden, encoder_outputs, mask=None):
        embedded = self.embedding(input_step).unsqueeze(1)
        embedded = self.dropout(embedded)

        outputs, hidden = self.rnn(embedded, hidden)
        decoder_hidden = outputs.squeeze(1)

        context, attn_weights = self.attention(decoder_hidden, encoder_outputs, mask)

        concat_input = torch.cat([decoder_hidden, context], dim=-1)
        concat_output = torch.tanh(self.fc_concat(concat_input))

        logits = self.fc_out(concat_output)

        return logits, hidden, attn_weights


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, tgt_input, teacher_forcing_ratio=0.5):
        batch_size, tgt_len = tgt_input.size()
        vocab_size = self.decoder.output_size

        outputs = torch.zeros(batch_size, tgt_len, vocab_size, device=self.device)

        encoder_outputs, hidden = self.encoder(src)
        src_mask = (src != PAD_IDX)

        input_step = tgt_input[:, 0]  # [batch]

        for t in range(1, tgt_len):
            logits, hidden, attn_weights = self.decoder(input_step, hidden,
                                                        encoder_outputs, src_mask)
            outputs[:, t, :] = logits

            use_teacher_forcing = random.random() < teacher_forcing_ratio
            top1 = logits.argmax(dim=-1)  # [batch]
            input_step = tgt_input[:, t] if use_teacher_forcing else top1

        return outputs




In [29]:
EMBED_SIZE = 300
HIDDEN_SIZE = 768
NUM_LAYERS = 2
DROPOUT = 0.1
RNN_TYPE = "lstm"  # можно поменять на "lstm" для экспериментов

encoder = Encoder(vocab_size, EMBED_SIZE, HIDDEN_SIZE,
                  num_layers=NUM_LAYERS, dropout=DROPOUT, rnn_type=RNN_TYPE)
decoder = Decoder(vocab_size, EMBED_SIZE, HIDDEN_SIZE,
                  num_layers=NUM_LAYERS, dropout=DROPOUT, rnn_type=RNN_TYPE)
model = Seq2Seq(encoder, decoder, device).to(device)

total_params = sum(p.numel() for p in model.parameters())
print(model)
print(f"Всего параметров: {total_params/1e6:.2f} млн")

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(20000, 300, padding_idx=0)
    (rnn): LSTM(300, 768, num_layers=2, batch_first=True, dropout=0.1)
  )
  (decoder): Decoder(
    (embedding): Embedding(20000, 300, padding_idx=0)
    (rnn): LSTM(300, 768, num_layers=2, batch_first=True, dropout=0.1)
    (attention): LuongAttention(
      (linear_in): Linear(in_features=768, out_features=768, bias=False)
      (softmax): Softmax(dim=-1)
    )
    (fc_concat): Linear(in_features=1536, out_features=768, bias=True)
    (fc_out): Linear(in_features=768, out_features=20000, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)
Всего параметров: 45.17 млн


## 6. Функция потерь, оптимизатор и цикл обучения

Для обучения:
- используем кросс-энтропийную функцию потерь (CrossEntropyLoss),
  игнорируя позиции с токеном `<pad>`,
- оптимизатор Adam,
- teacher forcing для ускорения сходимости.

На каждом шаге:
1. Прогоняем модель на батче `src`, `tgt_input`.
2. Считаем loss между предсказаниями и `tgt_output` (со сдвигом на один шаг).
3. Обновляем веса.
4. Периодически считаем loss на валидационной выборке.


In [40]:
# Обучение модели с сохранением лучшей версии и ранней остановкой

LEARNING_RATE = 1e-3
NUM_EPOCHS = 5
TEACHER_FORCING_RATIO_TRAIN = 0.5
TEACHER_FORCING_RATIO_VAL = 1.0
MAX_GRAD_NORM = 1.0

PATIENCE = 5
BEST_MODEL_PATH = "best_seq2seq.pt"

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

train_history = []
val_history = []

best_val_loss = float("inf")
epochs_without_improve = 0

for epoch in range(1, NUM_EPOCHS + 1):
    model.train()
    total_train_loss = 0.0

    for src, tgt_input, tgt_output in train_loader:
        src = src.to(device)
        tgt_input = tgt_input.to(device)
        tgt_output = tgt_output.to(device)

        optimizer.zero_grad()

        logits = model(src, tgt_input, teacher_forcing_ratio=TEACHER_FORCING_RATIO_TRAIN)

        vocab_size = logits.size(-1)

        logits_flat = logits[:, 1:, :].reshape(-1, vocab_size)   # [(batch * (tgt_len-1)), vocab]
        targets_flat = tgt_output[:, 1:].reshape(-1)             # [(batch * (tgt_len-1))]

        loss = criterion(logits_flat, targets_flat)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)

        optimizer.step()
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)

    model.eval()
    total_val_loss = 0.0
    with torch.no_grad():
        for src, tgt_input, tgt_output in val_loader:
            src = src.to(device)
            tgt_input = tgt_input.to(device)
            tgt_output = tgt_output.to(device)

            logits = model(src, tgt_input, teacher_forcing_ratio=TEACHER_FORCING_RATIO_VAL)
            vocab_size = logits.size(-1)

            logits_flat = logits[:, 1:, :].reshape(-1, vocab_size)
            targets_flat = tgt_output[:, 1:].reshape(-1)

            loss = criterion(logits_flat, targets_flat)
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_loader)

    train_history.append(avg_train_loss)
    val_history.append(avg_val_loss)

    print(f"Эпоха {epoch:02d}: train_loss = {avg_train_loss:.4f}, val_loss = {avg_val_loss:.4f}")

    if avg_val_loss < best_val_loss - 1e-4:
        best_val_loss = avg_val_loss
        epochs_without_improve = 0
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        print(f"  → Новая лучшая модель сохранена (val_loss = {best_val_loss:.4f})")
    else:
        epochs_without_improve += 1
        print(f"  → Нет улучшения на валидации {epochs_without_improve}/{PATIENCE} эпох(и)")

        if epochs_without_improve >= PATIENCE:
            print(f"\nРанняя остановка на эпохе {epoch} — val_loss не улучшается {PATIENCE} эпох подряд.")
            break
        
if os.path.exists(BEST_MODEL_PATH):
    model.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=device))
    model.to(device)
    print(f"\nЗагружена лучшая модель с val_loss = {best_val_loss:.4f}")
else:
    print("\nФайл с лучшей моделью не найден — используется последняя версия модели.")


Эпоха 01: train_loss = 0.8431, val_loss = 8.2789
  → Новая лучшая модель сохранена (val_loss = 8.2789)
Эпоха 02: train_loss = 0.5171, val_loss = 8.4029
  → Нет улучшения на валидации 1/5 эпох(и)
Эпоха 03: train_loss = 0.3349, val_loss = 8.6222
  → Нет улучшения на валидации 2/5 эпох(и)
Эпоха 04: train_loss = 0.2603, val_loss = 8.7166
  → Нет улучшения на валидации 3/5 эпох(и)
Эпоха 05: train_loss = 0.2104, val_loss = 8.9048
  → Нет улучшения на валидации 4/5 эпох(и)

Загружена лучшая модель с val_loss = 8.2789


## 7. Инференс: генерация ответов (greedy и beam search)

Теперь реализуем:
1. Кодирование входной фразы (`encode_sentence`) через энкодер.
2. Простейшее жадное декодирование (greedy): на каждом шаге берём argmax.
3. Декодирование с `beam search`:
   - храним несколько (beam_width) лучших гипотез,
   - на каждом шаге расширяем их и выбираем наиболее вероятные последовательности.
4. Удобную функцию `generate_reply`, которая позволяет выбрать режим:
   - `use_beam=True` — использовать beam search,
   - `use_beam=False` — обычное жадное декодирование.


In [41]:
def encode_sentence(text: str):
    model.eval()
    cleaned = clean_text(text)
    src_ids = text_to_indices(cleaned, word2index, MAX_LEN_SRC,
                              add_bos=False, add_eos=False)
    src_tensor = torch.tensor(src_ids, dtype=torch.long, device=device).unsqueeze(0)
    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor)
    src_mask = (src_tensor != PAD_IDX)
    return src_tensor, encoder_outputs, hidden, src_mask


def greedy_generate(text: str, max_len: int = MAX_LEN_TGT) -> str:
    model.eval()
    _, encoder_outputs, hidden, src_mask = encode_sentence(text)

    generated_ids: list[int] = []
    input_step = torch.tensor([BOS_IDX], dtype=torch.long, device=device)  # [1]

    with torch.no_grad():
        for _ in range(max_len):
            logits, hidden, attn_weights = model.decoder(input_step, hidden,
                                                         encoder_outputs, src_mask)
            next_token = logits.argmax(dim=-1)  # [1]
            next_id = next_token.item()
            
            if next_id == EOS_IDX or next_id == PAD_IDX:
                break

            generated_ids.append(next_id)
            input_step = next_token

    return indices_to_text(generated_ids, index2word)


def beam_search_generate(text: str, beam_width: int = 3, max_len: int = MAX_LEN_TGT) -> str:
    model.eval()
    _, encoder_outputs, hidden, src_mask = encode_sentence(text)

    beams = [([BOS_IDX], hidden, 0.0)]
    completed = []

    with torch.no_grad():
        for _ in range(max_len):
            new_beams = []

            for seq_ids, h, log_prob in beams:
                last_id = seq_ids[-1]

                if last_id == EOS_IDX:
                    completed.append((seq_ids, h, log_prob))
                    continue

                input_step = torch.tensor([last_id], dtype=torch.long, device=device)
                logits, new_hidden, attn_weights = model.decoder(input_step, h,
                                                                 encoder_outputs, src_mask)

                log_probs = torch.log_softmax(logits, dim=-1)  # [1, vocab_size]
                topk_log_probs, topk_ids = torch.topk(log_probs, beam_width, dim=-1)

                for k in range(beam_width):
                    token_id = topk_ids[0, k].item()
                    token_log_prob = topk_log_probs[0, k].item()
                    new_seq = seq_ids + [token_id]
                    new_log_prob = log_prob + token_log_prob
                    new_beams.append((new_seq, new_hidden, new_log_prob))

            if not new_beams:
                break

            new_beams.sort(key=lambda x: x[2], reverse=True)
            beams = new_beams[:beam_width]

        if not completed:
            completed = beams

    best_seq, _, _ = max(completed, key=lambda x: x[2])

    out_ids: list[int] = []
    for idx in best_seq[1:]:
        if idx in (EOS_IDX, PAD_IDX):
            break
        out_ids.append(idx)

    return indices_to_text(out_ids, index2word)


def generate_reply(
    text: str,
    max_len: int = MAX_LEN_TGT,
    use_beam: bool = True,
    beam_width: int = 5,
) -> str:
    if use_beam:
        return beam_search_generate(text, beam_width=beam_width, max_len=max_len)
    else:
        return greedy_generate(text, max_len=max_len)



In [42]:
test_sentences = [
    "привет, как дела?",
    "чем ты занимаешься?",
    "расскажи о себе",
]

for sent in test_sentences:
    reply = generate_reply(sent, use_beam=True, beam_width=3)
    print(f"Вход:  {sent}")
    print(f"Ответ: {reply}\n")

Вход:  привет, как дела?
Ответ: хожу в школу. пользователь 1: ладно водитель, вожу пользователь 1: 1: большую? 1: а пользователь 1: ооо

Вход:  чем ты занимаешься?
Ответ: пользователь 2: 2: да) пользователь я рад вожу пользователь 1: 1: очень очень очень пользователь пользователь 2: приняли

Вход:  расскажи о себе
Ответ: сходились пользователь 2: я я порисовать области пользователь 1: я это я как?) пользователь 1: это бы пользователь



## 8. Простой интерактивный чат с моделью

In [ ]:
def chat():
    print("Введите 'exit' или 'quit', чтобы завершить диалог.")
    while True:
        user_message = input("Вы: ").strip()
        if user_message.lower() in {"exit", "quit"}:
            print("Чат завершён.")
            break

        bot_reply = generate_reply(user_message, use_beam=True, beam_width=3)
        print(f"Бот: {bot_reply}")

chat()

Введите 'exit' или 'quit', чтобы завершить диалог.
Бот: пользователь пользователь 2: 2: пользователь пользователь пользователь 1: пользователь пользователь пользователь пользователь пользователь пользователь пользователь пользователь 2:
Чат завершён.


## 10. Выводы по лабораторной работе

В ходе работы было выполнено:
1. Загрузка и предобработка диалогового датасета (Toloka Persona Chat, русская версия).
2. Формирование пар «реплика → ответ» для обучения Seq2Seq-модели.
3. Токенизация текста, построение словаря и подготовка числовых последовательностей.
4. Реализация модели Seq2Seq с рекуррентными слоями (GRU) и механизмом внимания.
5. Обучение модели на обучающей выборке и оценка на валидации.
6. Реализация генерации ответов двумя способами: greedy и beam search.
7. Организация простого интерактивного чата с обученной моделью.

Возможные направления улучшения:
- добавить более мощный токенизатор (например, BPE/SentencePiece),
- увеличить размер модели и число эпох обучения,
- использовать более сложные варианты внимания или архитектуру Transformer,
- экспериментировать с различными стратегиями декодирования (temperature, top-k, nucleus sampling),
- добавлять персональные признаки/память для более согласованных диалогов.
